# Fine Tuning of the convolutional network

In [1]:
%pylab
%matplotlib inline

import glob
import os
import mne
CORPORA_PATH = "~/corpora/sets"

file_path = os.path.expanduser(CORPORA_PATH)
files = glob.glob(os.path.join(file_path, "*.set"))

def normalize_subject(X):
    mean = X.mean(axis=(0, 2)).reshape(-1, 1)
    std = X.std(axis=(0, 2)).reshape(-1, 1)
    return (X - mean) / std

def load_data(filename, normalize=True):
    data_mne = mne.io.read_raw_eeglab(filename, preload=True, event_id={"0": 1, "1": 2})
    data_mne.filter(0, 20)
    events = mne.find_events(data_mne)
    epochs = mne.Epochs(
        data_mne, events,
        baseline=(None, 0), tmin=-0.1, tmax=0.7)

    epochs.load_data()
    
    ch_names = epochs.ch_names
    
    X = epochs.get_data()[:, :-1]
    y = (events[:, 2] == 2).astype('float')

    if len(events) != len(epochs):
        raise ValueError("Epochs events mismatch")
    if normalize: 
        X = normalize_subject(X)
    X = X[..., np.newaxis]
    
    return X, y 


Using matplotlib backend: TkAgg
Populating the interactive namespace from numpy and matplotlib


In [2]:
from keras.models import load_model
channels = ['AF3', 'F7', 'F3', 'FC5', 'T7', 'P7', 'O1', 'O2', 'P8', 'T8', 'FC6', 'F4', 'F8', 'AF4', 'STI 014']

model = load_model("models/model.h5")

Using TensorFlow backend.
/home/jmperez/.pyenv/versions/3.6.0/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [5]:
X, y = load_data(files[143])

Reading /home/jmperez/corpora/sets/PruebasMuseo_358001.fdt
Reading 0 ... 63231  =      0.000 ...   493.992 secs...
Setting up low-pass filter at 20 Hz
h_trans_bandwidth chosen to be 5.0 Hz
Filter length of 169 samples (1.320 sec) selected
1800 events found
Events id: [1 2]
1800 matching events found
0 projection items activated
Loading data for 1800 events and 104 original time points ...
0 bad epochs dropped


In [6]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test= train_test_split(X, y, test_size=0.1, stratify=y)

In [7]:


from sklearn.metrics import precision_score, recall_score, roc_auc_score, accuracy_score

def print_metrics(model, X_test, y_test):
    y_pred = model.predict_classes(X_test)
    y_prob = model.predict(X_test)

    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    auc = roc_auc_score(y_test, y_prob)
    accuracy = accuracy_score(y_test, y_pred)

    print("""
    Accuracy   = {}
    Precision  = {}
    Recall     = {}
    ROC AUC    = {}
    """.format(accuracy, precision, recall, auc))
    
print_metrics(model, X_test, y_test)


    Accuracy   = 0.5555555555555556
    Precision  = 0.20238095238095238
    Recall     = 0.5666666666666667
    ROC AUC    = 0.589111111111111
    


In [8]:
model.layers

Let's fix the first two convolutional layers

In [20]:

for i in range(4):
    model.layers[i].trainable = False



model.compile(loss='binary_crossentropy', # using the cross-entropy loss function
              optimizer='rmsprop', 
              metrics=['accuracy']) # reporting the accuracy
[(l, "Trainable: {}".format(l.trainable)) for l in model.layers]

[(<keras.layers.convolutional.Conv2D at 0x7f2862607ef0>, 'Trainable: False'),
 (<keras.layers.convolutional.Conv2D at 0x7f2860a3c240>, 'Trainable: False'),
 (<keras.layers.core.Flatten at 0x7f2860a3c160>, 'Trainable: False'),
 (<keras.layers.core.Dropout at 0x7f286076c4a8>, 'Trainable: False'),
 (<keras.layers.core.Dense at 0x7f28607d5ba8>, 'Trainable: True'),
 (<keras.layers.core.Dense at 0x7f286073c860>, 'Trainable: True')]

In [21]:
from keras.callbacks import ModelCheckpoint, EarlyStopping
checkpointer = ModelCheckpoint(filepath='model.h5', verbose=1, save_best_only=True)
early_stopping = EarlyStopping(monitor='val_loss', patience=3)

model.fit(
    X_train, y_train, epochs=30, 
    batch_size=64, class_weight={0:1, 1:6}, validation_split=0.01,
    callbacks=[checkpointer, early_stopping]
)

ValueError: Error when checking input: expected conv2d_3_input to have 4 dimensions, but got array with shape (1620, 87360)

In [11]:
print_metrics(model, X_test, y_test)


    Accuracy   = 0.5388888888888889
    Precision  = 0.20224719101123595
    Recall     = 0.6
    ROC AUC    = 0.6077777777777778
    


In [12]:
np.hstack([model.predict(X_test).reshape(-1, 1), y_test.reshape(-1, 1)])

array([[ 0.49438933,  0.        ],
       [ 0.59221601,  1.        ],
       [ 0.44325417,  0.        ],
       [ 0.49404421,  0.        ],
       [ 0.54224604,  0.        ],
       [ 0.48226279,  0.        ],
       [ 0.78814924,  1.        ],
       [ 0.57949287,  0.        ],
       [ 0.49402833,  0.        ],
       [ 0.53376693,  0.        ],
       [ 0.55322862,  0.        ],
       [ 0.49195325,  0.        ],
       [ 0.45938343,  0.        ],
       [ 0.39896855,  0.        ],
       [ 0.44019067,  1.        ],
       [ 0.65102875,  0.        ],
       [ 0.50291926,  0.        ],
       [ 0.46069604,  0.        ],
       [ 0.49177676,  0.        ],
       [ 0.45464355,  0.        ],
       [ 0.45160869,  0.        ],
       [ 0.56659019,  0.        ],
       [ 0.47286335,  0.        ],
       [ 0.6067099 ,  0.        ],
       [ 0.4832212 ,  0.        ],
       [ 0.56725192,  1.        ],
       [ 0.42826706,  0.        ],
       [ 0.46378893,  1.        ],
       [ 0.4404662 ,

## Training an SVM

In [13]:
from sklearn.svm import SVC

clf = SVC(probability=True, C=3)
X_tr = X_train.reshape(X_train.shape[0], -1)
X_te = X_test.reshape(X_test.shape[0], -1)
clf.fit(X_tr, y_train)

SVC(C=3, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=True, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [14]:
y_pred = clf.predict(X_te)
y_prob = clf.predict_proba(X_te)[:, 0]

precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
auc = roc_auc_score(y_test, y_prob)
accuracy = accuracy_score(y_test, y_pred)

print("""
Accuracy   = {}
Precision  = {}
Recall     = {}
ROC AUC    = {}
""".format(accuracy, precision, recall, auc))


Accuracy   = 0.8333333333333334
Precision  = 0.0
Recall     = 0.0
ROC AUC    = 0.41555555555555557



/home/jmperez/.pyenv/versions/3.6.0/envs/p300/lib/python3.6/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


## Training SVM using features from the conv net

In [15]:
from keras import Model

model = load_model("models/model.h5")

print(model.layers)



[<keras.layers.convolutional.Conv2D object at 0x7f2862607ef0>, <keras.layers.convolutional.Conv2D object at 0x7f2860a3c240>, <keras.layers.core.Flatten object at 0x7f2860a3c160>, <keras.layers.core.Dropout object at 0x7f286076c4a8>, <keras.layers.core.Dense object at 0x7f28607d5ba8>, <keras.layers.core.Dense object at 0x7f286073c860>]


In [16]:
intermediate_layer_model = Model(inputs=model.input,
                                 outputs=model.layers[2].output)

new_X = intermediate_layer_model.predict(X)

new_X = new_X.reshape(new_X.shape[0], -1)

In [17]:

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test= train_test_split(new_X, y, test_size=0.1, stratify=y)


In [18]:
from sklearn.svm import LinearSVC

clf = LinearSVC(C=2, class_weight="balanced")


clf.fit(X_train, y_train)

LinearSVC(C=2, class_weight='balanced', dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

In [19]:
y_pred = clf.predict(X_test)

precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)

accuracy = accuracy_score(y_test, y_pred)

print("""
Accuracy   = {}
Precision  = {}
Recall     = {}
""".format(accuracy, precision, recall))


Accuracy   = 0.7388888888888889
Precision  = 0.2702702702702703
Recall     = 0.3333333333333333

